In [8]:
###############################################################################
# DOMINIO TESIS - COLADA CONTINUA DE DOBLE RODILLO
###############################################################################

using Gridap

# ==============================
# PARÁMETROS GEOMÉTRICOS
# ==============================
R  = 0.25         # Radio del rodillo [m]
H  = 0.125        # Longitud vertical de la lámina [m]
W2 = 0.6725       # Mitad del ancho total [m]
tol = 1e-6  #tolerancia numerica (base)


# Dominio en coordenadas globales
# Origen en la parte superior del rodillo izquierdo
# Eje X → derecha, Eje Y → hacia abajo
x_min = 0.0
x_max = R + W2     # desde el borde del rodillo hasta el centro de la lámina
y_min = 0.0
y_max = H + R       # altura total (rodillo + lámina)

# ==============================
# CREAR DOMINIO Y MALLADO
# ==============================
# Definimos el dominio rectangular que contiene todo
domain = (x_min, x_max, y_min, y_max)

# Discretización (resolución de la malla)(PROBANDO POR SI DESPUES LE PONEMOS MAS O MENOS)
nx = 60   # elementos en x
ny = 40   # elementos en y

# Modelo discreto (malla cartesiana)
model = CartesianDiscreteModel(domain, (nx, ny))




println("FUNCIONA, DOMINIO MEJORADO")
println(" Etiqueta: 0 interiore, 1 rodillo x = 0, simetria x = R, 3 top y = 0, 4 botton (y = H)")



println("✅ Dominio generado correctamente:")
println("   X ∈ [$(x_min), $(x_max)]")
println("   Y ∈ [$(y_min), $(y_max)]")
println("   Malla con $(nx) x $(ny) celdas")

# ==============================
# VISUALIZAR (PRUEBA 1 DEL PARAVIEW)
# ==============================
writevtk(model, "dominio_simplificado")
println("📦 Malla exportada a dominio_simplificado.vtu (abre en ParaView o GridapMakie)")
println("FUNCIONA HASTA AQUI")


FUNCIONA, DOMINIO MEJORADO
 Etiqueta: 0 interiore, 1 rodillo x = 0, simetria x = R, 3 top y = 0, 4 botton (y = H)
✅ Dominio generado correctamente:
   X ∈ [0.0, 0.9225]
   Y ∈ [0.0, 0.375]
   Malla con 60 x 40 celdas
📦 Malla exportada a dominio_simplificado.vtu (abre en ParaView o GridapMakie)
FUNCIONA HASTA AQUI


In [7]:
using Gridap
using LinearAlgebra

# === Parámetros del dominio
R  = 0.25
H  = 0.125
W2 = 0.6725
x_min = 0.0
x_max = R + W2
y_min = 0.0
y_max = H + R

# === Crear triangulación del modelo existente
Ω = Triangulation(model)

# Obtener coordenadas de las celdas (centroides)
cell_coords = get_cell_coordinates(Ω)

# === Inicializar etiquetas
btag = zeros(Int, size(cell_coords,1))

# === Asignar etiquetas según posición (comparando centroides)
for i in 1:size(cell_coords,1)
    x = cell_coords[i,1]
    y = cell_coords[i,2]

    if abs(x - 0.0) < 1e-3
        btag[i] = 1   # Rodillo (borde izquierdo)
    elseif abs(x - R) < 1e-3
        btag[i] = 2   # Simetría (borde derecho)
    elseif abs(y - 0.0) < 1e-3
        btag[i] = 3   # Entrada (parte superior)
    elseif abs(y - H) < 1e-3
        btag[i] = 4   # Salida (parte inferior)
    else
        btag[i] = 0   # Interior
    end
end

# === Crear campo de celdas y exportar para ParaView
cell_field = CellField(btag, Ω)
writevtk(Ω, "dominio_etiquetado", cellfields = ["boundary_tag" => cell_field])

println("✅ Archivo 'dominio_etiquetado.vtu' generado correctamente.")
println("   Etiquetas: 0 = interior, 1 = rodillo (x≈0), 2 = simetría (x≈R), 3 = entrada (y≈0), 4 = salida (y≈H).")


LoadError: MethodError: no method matching -(::Vector{VectorValue{2, Float64}}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar
The function `-` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  -([91m::Missing[39m, ::Number)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:123[24m[39m
[0m  -([91m::BigFloat[39m, ::Union{Float16, Float32, Float64})
[0m[90m   @[39m [90mBase[39m [90m[4mmpfr.jl:565[24m[39m
[0m  -([91m::Complex{Bool}[39m, ::Real)
[0m[90m   @[39m [90mBase[39m [90m[4mcomplex.jl:329[24m[39m
[0m  ...
